## Desáfio de Código Datenworks

### Carregando os dados e entendo seus desáfios 

Primeiramente irei carregar os arquivos .csv e os transformarei em um Data Frame do Pandas.

Irei exibir esses dados e assim procurar por insight de como resolver o problema proposto

In [1]:
import pandas as pd

In [2]:
df_1= pd.read_csv('https://raw.githubusercontent.com/Datenworks/developer-code-challenge/master/summoner_challenge/files/dataset_1.csv')

In [3]:
df_1

,NAME,ATTACK_SPEED,ARMOR
0,Draven,0.679,29
1,Ezreal,0.625,22
2,Caitlyn,0.568,28
3,Varus,0.658,27
4,Kai'Sa,0.644,28
5,Kog'Maw,0.665,24
6,Jinx,0.625,28
7,Quinn,0.668,28
8,Tristana,0.656,26
9,Miss Fortune,0.656,28


In [4]:
df_2= pd.read_csv('https://raw.githubusercontent.com/Datenworks/developer-code-challenge/master/summoner_challenge/files/dataset_2.csv')

In [5]:
df_2

,NAME,ATTACK_DAMAGE,MAGIC_RESIST
0,Draven,60,30
1,Jinx,57,30
2,Kalista,67,30
3,Ashe,61,30
4,Caitlyn,58,30
5,Kai'Sa,59,30
6,Corki,60,30
7,Tristana,61,30
8,Twitch,59,30


 Como é possivel observar nos data frames exibidos será nescessario unir os dois arquivos, ao mesmo tempo são observadas duas colunas desnessarias.

A partir dos Data Frames foi possível  inferir algumas soluções.

- Eliminar colunas desnecessarias: as colunas relacionadas com atributos defensivos serão removidas para melhorar a performance 

- Como a maior parte dos processos aqui necessitam do nome das colunas (labels) estejam padronizados sera feita verificação

- Sera necessario unir os 2 data frames

- calcular o DPS

- Imprimir ordenado decresente em relação ao DPS 

## Implementando Classes

In [6]:
# Classe com a função de fazer toda e qualquer transformação no DataFrame

class Transforma():
  
    def __init__(self, df):
        self.df = df
    
        
    def tudo_maiuscula(self):
        a= list(self.df.columns)
        
        for x in range(len(a)):
            a[x] = a[x].upper()
    
            self.df.columns = a
        
# Função que remove todas as colunas que não serão usadas para melhorar o desempenho
# Recebe do Fiscal as colunas corretas e corta as outras

    def remove_colunas_inuteis(self):

        fiscal = Fiscal(self.df)

        ajudante = fiscal.delvolve_oque_esta_certo()

        self.df = self.df.loc[:,ajudante]
        
# Calcula o DPS e devolve o DF apenas com as colunas NAME e DPS
        
    def dps(self):
        self.df['DPS'] = self.df['ATTACK_DAMAGE']*self.df['ATTACK_SPEED']
        self.df = self.df.loc[:,['NAME','DPS']]
           

In [7]:
# Classe Concerta tem a função de concertar qualquer erro humano, se por algum motivo o nome da coluna entrar com um nome
#diferente a aplicação ira quebrar. 

class Concerta():
        
# Função que arruma os nomes caso ele esteja em uma lista pré-definida de erros comuns        
 
    def nomes_parecidos(self,valor_colunas, valor_esperado, possiveis_inputs_errados):
        
        for x in possiveis_inputs_errados:
            if x in valor_colunas:
                a = valor_colunas.index(x)
                valor_colunas[a] = valor_esperado
                return valor_colunas
        return valor_colunas
        

In [8]:
# Classe constituição onde estão as leis da aplicação, ela guarda um dicionario onde as chaves são os valores que podemos
#encontrar na aplicação ( valores corretos) e seus valores serão usados pela classe concerta para concertar

class Constituicao():

     def __init__(self):
        
        name = ['NOME','NOMES', 'NAMES'] 
        atk_dam = ['ATK_DAMAGE','ATKDAMAGE', 'DAMAGE']
        atk_speed = ['ATK_SPEED','ATKSPEED', 'SPEED']
        
        self._colunas_permitidas = {'NAME':name, 'ATTACK_DAMAGE':atk_dam, 'ATTACK_SPEED':atk_speed}      
    
        @property
        def colunas_permitidas(self):
            return self._colunas_permitidas

In [9]:
# Responsavel por fiscalizar se o nome das colunas está correto

class Fiscal():
  
    def __init__(self, df):
        self.df = df
        self.valor_colunas = list(self.df.columns)
        
        
# Devolve apenas os valores da constituição presentes nas colunas, caso não esteja correto (colunas faltantes ou nomes errados)
#lança uma execeção 
        
    def delvolve_oque_esta_certo (self):
        ajudante = []
        leis = Constituicao()

        for x in leis._colunas_permitidas:

            if self.garante_coluna_correta( x , leis._colunas_permitidas[x]) == True:
                ajudante.append(x)
                if len(ajudante)==2:
                    return ajudante
        raise ValueError('Colunas incompativeis, checar seus nomes')
        
# Garante que um valor esperado está nos nomes das colunas, caso contrario tenta concertar

    def garante_coluna_correta(self, valor_esperado: str, possiveis_inputs_errados: list):
        
        if self.fiscaliza(valor_esperado) == True:
            return True
        
        conserta = Concerta()
        coluna_colunas = conserta.nomes_parecidos(self.valor_colunas, valor_esperado, possiveis_inputs_errados) 
        
        if self.fiscaliza(valor_esperado) == True:
            self.df.columns = self.valor_colunas
            return True

        return False

    def fiscaliza(self,valor_esperado):

        if valor_esperado in self.valor_colunas:
            return True
        else:
            return False

## Implementação

In [10]:
trans_1 = Transforma(df_1)
trans_2 = Transforma(df_2)

trans_1.tudo_maiuscula()
trans_2.tudo_maiuscula()

trans_1.remove_colunas_inuteis()
trans_2.remove_colunas_inuteis()



# Faz um merge usando a coluna NAME
trans_1.df = trans_1.df.merge(trans_2.df, on='NAME')

trans_1.dps()

# Organiza do mais forte para o mais fraco
trans_1.df.sort_values('DPS', ascending = False ,inplace = True )

print(trans_1.df)

       NAME     DPS
0    Draven  40.740
4  Tristana  40.016
2    Kai'Sa  37.996
3      Jinx  35.625
1   Caitlyn  32.944


## Testes

In [11]:
from unittest import TestCase

In [12]:
class TestTransforma(TestCase):

    def test_deve_retornar_todas_as_palavras_da_lista_maiusculas(self,df_1):

        df_1.columns = ['NAME', 'ATTACK_SPEED', 'Armor']

        t = Transforma(df_1)

        t.tudo_maiuscula()

        valor_esperado = ['NAME', 'ATTACK_SPEED', 'ARMOR']
        
        import operator
        a = all( map(operator.eq, df_1.columns ,  valor_esperado ) )
        
        self.assertEqual(True, a)


In [13]:
t = TestTransforma()

t.test_deve_retornar_todas_as_palavras_da_lista_maiusculas(df_1)

In [14]:
class TestFiscal(TestCase):

    def test_funcao_garante_coluna_correta(self,df_1, lista):

        df_1.columns = lista
        
        transforma = Transforma(df_1)
        
        transforma.tudo_maiuscula()

        f = Fiscal(df_1)

        f.garante_coluna_correta('NAME',['NAMES','NOMES'])
        

        valor_esperado = ['NAME', 'ATTACK_SPEED', 'ARMOR']
        
        import operator
        a = all( map(operator.eq, df_1.columns ,  valor_esperado ) )
        
        self.assertEqual(True, a)
        

In [15]:
t = TestFiscal()

# Testando se ao receber um nome correto ele retorna true ( e testanto a função tudo maiuscula junto)

t.test_funcao_garante_coluna_correta(df_1, ['NAMEs', 'ATTACK_SPEED', 'Armor'])

In [16]:
# Testando se ao receber um dado errado mas que ela deve conseguir concertar se realmente concerta

t.test_funcao_garante_coluna_correta(df_1, ['NOMES', 'ATTACK_SPEED', 'Armor'])

In [17]:
# Testando se ao receber um dado errado que não deve conseguir concertar se devolve um erro

t.test_funcao_garante_coluna_correta(df_1, ['NOMES', 'ATTACK_SPEED', 'ArmoOr'])

AssertionError: True != False